In [1]:
import tensorflow as tf
import os
import sys
from ssd_network import *
add_dir = os.path.abspath('./data/')
sys.path.append(add_dir)
import numpy as np
import cv2
import tensorflow.contrib.slim as slim
from tensorflow.python.ops import random_ops,control_flow_ops
from tensorflow.python.keras.initializers import he_normal

import random
from data import ssd_common
from data import loss_function
from matplotlib import pyplot as plt
from tensorflow.python.framework.graph_util import convert_variables_to_constants
from set_flags_variables import *
from util import *
from ssd import *
from loss import *
from ResneXt_modify import *

In [2]:
#####################the start of bottle neck##################
inputs = tf.placeholder(tf.float32, [None, FLAGS.image_size, FLAGS.image_size, 3],name='inputs')
labels = tf.placeholder(tf.int32, [None,None])
bboxes = tf.placeholder(tf.float32, [None,None,4])
# keep_prob = tf.placeholder(tf.float32)
is_training = tf.placeholder(tf.bool,name='is_training')
#####resnext######################
# resnext = ResNeXt(inputs,is_training)
# end_points = resnext.model
#################################
####vgg16 backbone##
end_points = vgg16_backbone(inputs,is_training)
####################################
predictions =[]
logits = []
localisations=[]
for i,layer in enumerate(feat_layers):
    with tf.variable_scope(layer + '_box'):
        p, l = ssd_multibox_layer(end_points[layer],anchor_sizes[i],anchor_ratios[i],feat_shapes[i],normalizations[i])
        predictions.append(tf.contrib.slim.softmax(p))
        logits.append(p)########(layers,batch_size,featuremap,featuremap,boxes_layer,num_class)
        localisations.append(l)#####(layers,batch_size,featuremap,featuremap,boxes_layer,4)
        
# generate bboxes for each feet layer
anchor_layers_original = ssd_anchors_all_layers((FLAGS.image_size,FLAGS.image_size),
                           feat_shapes,
                           anchor_sizes,
                           anchor_ratios,
                           anchor_steps,
                           offset=0.5,
                           dtype=np.float32)

################     encode bboxes to each layer      #################
feat_labels,feat_localizations,feat_scores = encode_bboxes(labels,bboxes,anchor_layers_original)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [3]:
# def ssd_losses(logits, localisations,
#                gclasses, glocalisations, gscores,
#                match_threshold=0.5,
#                negative_ratio=3.,
#                alpha=1.,
#                label_smoothing=0.,
#                device='/cpu:0',
#                scope=None):
#     with tf.name_scope(scope, 'ssd_losses'):
#         lshape = tfe.get_shape(logits[0], 5)
#         num_classes = lshape[-1]
#         batch_size = lshape[0]

#         # Flatten out all vectors!
#         flogits = []
#         fgclasses = []
#         fgscores = []
#         flocalisations = []
#         fglocalisations = []
#         for i in range(len(logits)):
#             flogits.append(tf.reshape(logits[i], [-1, num_classes]))
#             fgclasses.append(tf.reshape(gclasses[i], [-1]))
#             fgscores.append(tf.reshape(gscores[i], [-1]))
#             flocalisations.append(tf.reshape(localisations[i], [-1, 4]))
#             fglocalisations.append(tf.reshape(glocalisations[i], [-1, 4]))
#         # And concat the crap!
#         logits = tf.concat(flogits, axis=0)
#         gclasses = tf.concat(fgclasses, axis=0)
#         gscores = tf.concat(fgscores, axis=0)
#         localisations = tf.concat(flocalisations, axis=0)
#         glocalisations = tf.concat(fglocalisations, axis=0)
#         dtype = logits.dtype

#         # Compute positive matching mask...
#         pmask = gscores > match_threshold
#         #################################someone on github#########
# #         pmask = tf.concat(axis=0, values=[pmask[:tf.argmax(gscores)],[True],pmask[tf.argmax(gscores)+1:]])
#         ###########################################################
#         fpmask = tf.cast(pmask, dtype)
#         n_positives = tf.reduce_sum(fpmask)

#         # Hard negative mining...
#         no_classes = tf.cast(pmask, tf.int32)
#         predictions = slim.softmax(logits)
#         nmask = tf.logical_and(tf.logical_not(pmask),
#                                gscores > -0.5)
#         fnmask = tf.cast(nmask, dtype)
# #         nvalues = tf.where(nmask,
# #                            predictions[:, 0],
# #                            1. - fnmask)
# #         nvalues_flat = tf.reshape(nvalues, [-1])
# #         # Number of negative entries to select.
# #         max_neg_entries = tf.cast(tf.reduce_sum(fnmask), tf.int32)
# #         n_neg = tf.cast(negative_ratio * n_positives, tf.int32) + batch_size
# #         n_neg = tf.minimum(n_neg, max_neg_entries)

# #         val, idxes = tf.nn.top_k(-nvalues_flat, k=n_neg)
# #         max_hard_pred = -val[-1]
# #         # Final negative mask.
# #         nmask = tf.logical_and(nmask, nvalues < max_hard_pred)
# #         fnmask = tf.cast(nmask, dtype)

#         # Add cross-entropy loss.
#         with tf.name_scope('cross_entropy_pos'):
#             loss = focal_loss(y_pred=logits,y_true=gclasses)
#             loss = tf.div(tf.reduce_sum(loss * fpmask), tf.cast(batch_size,dtype), name='value')
#             tf.losses.add_loss(loss)
#             cross_entropy_pos_loss = loss
#         with tf.name_scope('cross_entropy_neg'):
#             loss = focal_loss(y_pred=logits,y_true=no_classes)
#             loss = tf.div(tf.reduce_sum(loss * fnmask), tf.cast(batch_size,dtype), name='value')
#             tf.losses.add_loss(loss)
#             cross_entropy_neg_loss = loss
#         # Add localization loss: smooth L1, L2, ...
#         with tf.name_scope('localization'):
#             # Weights Tensor: positive mask + random negative.
#             weights = tf.expand_dims(alpha * fpmask, axis=-1)
#             loss = custom_layers.abs_smooth(localisations - glocalisations)
#             loss = tf.div(tf.reduce_sum(loss * weights), tf.cast(batch_size,dtype), name='value')
#             tf.losses.add_loss(loss)
#             localization_loss = loss
#         return cross_entropy_pos_loss,cross_entropy_neg_loss,localization_loss

In [4]:
##################################This part is used for inference #########################
decoded_boxes = ssd_common.tf_ssd_bboxes_decode(localisations,
                         anchor_layers_original,
                         prior_scaling=[0.1, 0.1, 0.2, 0.2],
                         scope='ssd_bboxes_decode')

inference_network_part(decoded_boxes,logits,predictions)
#######################################################################################################################

In [5]:
# for i in range(FLAGS.batch_size):
#     index = (bboxes[i][...,2]-bboxes[i][...,0])>0
#     temp_label =[]
#     temp_box = []
#     for j in range(60):
#         if(index[j] == True):
#             temp_label.append(labels[j])
#             temp_box.append(bboxes[j])

In [6]:
###########need to be modified############################################
pos_loss,neg_loss,loc_loss = ssd_losses(logits, #预测类别
                   localisations,#预测位置
                   feat_labels, #ground truth 类别
                   feat_localizations, #ground truth 位置
                   feat_scores,#ground truth 分数
                   match_threshold=match_threshold,
                   negative_ratio=3.,
                   alpha=1.,
                   label_smoothing=0.,
                   scope='ssd_losses')
# add l2 regularization loss to the total losses
total_losses = pos_loss+neg_loss+loc_loss+ tf.losses.get_regularization_loss()

In [7]:
annotation = get_path_and_annotation('/media/xinje/New Volume/VOC07&12/VOC2007/train/train_OCR.txt')
train_iter = read_data(annotation,FLAGS.batch_size,is_random=False,is_crop=True)

In [8]:
tf.summary.scalar('scala/loss',total_losses)
tf.summary.scalar('scala/neg_loss',neg_loss)
tf.summary.scalar('scala/pos_loss',pos_loss)
tf.summary.scalar('scala/loc_loss',loc_loss)
tf.summary.scalar('scala/reg_loss',tf.losses.get_regularization_loss())
merged_summary_op = tf.summary.merge_all()
update_opts = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies([tf.group(*update_opts)]):
    train_opt = tf.train.AdamOptimizer(learning_rate).minimize(total_losses)

Instructions for updating:
Use tf.cast instead.


In [ ]:
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(FLAGS.log_dir,sess.graph)
    saver = tf.train.Saver(max_to_keep=10)
    sess.run(tf.global_variables_initializer())
    ckpt = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
    start_step = 0
    if ckpt:
        saver.restore(sess,ckpt)
        print('Restore from the checkpoint {0}'.format(ckpt))
        start_step += int(ckpt.split('-')[-1])
    else:
        print('Train ssd from start')
    for i in range(start_step,1000000):
        image_batch,annotation_batch,cls_batch = next(train_iter)      
        [_,train_summary,temp_loss,temp_pos_loss,temp_neg_loss,temp_loc_loss,temp_label,temp_score,temp_loc] = sess.run([train_opt,merged_summary_op,total_losses,pos_loss,neg_loss,loc_loss,feat_labels,feat_scores,feat_localizations],feed_dict={inputs:image_batch,bboxes:annotation_batch,labels:cls_batch,is_training: True})
        if(i%50==0):
            print('Batch: '+str(i)+'   Current total loss: '+str(temp_loss)+' pos_loss: '+str(temp_pos_loss)+' neg_loss: '+str(temp_neg_loss)+' loc_loss: '+str(temp_loc_loss))
        train_writer.add_summary(train_summary, i)
        if(i%500==0):
            #save model for training
            saver.save(sess,os.path.join(FLAGS.checkpoint_dir,'OCR-'),global_step=i) 
            ##########################save model for inference#############################################
            constant_graph = convert_variables_to_constants(sess, sess.graph_def, ['boxes','scores','classes'])
            with tf.gfile.FastGFile('./model.pb', mode='wb') as f:
                f.write(constant_graph.SerializeToString())
            ######################################################################
            print('Model saved !!!')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /media/xinje/New Volume/ssd/vgg16/hnm/OCR--576500
Restore from the checkpoint /media/xinje/New Volume/ssd/vgg16/hnm/OCR--576500
Batch: 576500   Current total loss: 8.733767 pos_loss: 3.1376 neg_loss: 1.6195034 loc_loss: 2.2930245
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 141 variables.
INFO:tensorflow:Converted 141 variables to const ops.
Instructions for updating:
Use tf.gfile.GFile.
Model saved !!!
Batch: 576550   Current total loss: 12.956655 pos_loss: 4.456753 neg_loss: 4.6945777 loc_loss: 2.121324
Batch: 576600   Current total loss: 11.193807 pos_loss: 3.8663871 neg_loss: 3.2960327 loc_loss: 2.3472512
Batch: 576650   Current total loss: 10.042488 pos_loss: 3.2717106 neg_loss: 2.005685 loc_loss: 3.0804787
Batch: 576700

Batch: 579500   Current total loss: 11.088468 pos_loss: 3.5935967 neg_loss: 2.74382 loc_loss: 3.0451674
INFO:tensorflow:Froze 141 variables.
INFO:tensorflow:Converted 141 variables to const ops.
Model saved !!!
Batch: 579550   Current total loss: 13.877703 pos_loss: 5.584546 neg_loss: 2.7192564 loc_loss: 3.8677742
Batch: 579600   Current total loss: 13.561625 pos_loss: 5.924998 neg_loss: 2.524359 loc_loss: 3.4060001
Batch: 579650   Current total loss: 4.884329 pos_loss: 1.2929316 neg_loss: 0.7784832 loc_loss: 1.1062624
Batch: 579700   Current total loss: 10.439279 pos_loss: 4.609523 neg_loss: 2.1635356 loc_loss: 1.9594234
Batch: 579750   Current total loss: 14.494701 pos_loss: 4.8974547 neg_loss: 4.925635 loc_loss: 2.9645393
Batch: 579800   Current total loss: 22.06076 pos_loss: 12.274958 neg_loss: 2.7591465 loc_loss: 5.319094
Batch: 579850   Current total loss: 11.408105 pos_loss: 3.382527 neg_loss: 4.044466 loc_loss: 2.273204
Batch: 579900   Current total loss: 13.436742 pos_loss: 7.

Batch: 582950   Current total loss: 10.10984 pos_loss: 3.8816395 neg_loss: 1.952927 loc_loss: 2.5467725
Batch: 583000   Current total loss: 16.857687 pos_loss: 6.7223687 neg_loss: 3.9863658 loc_loss: 4.420112
INFO:tensorflow:Froze 141 variables.
INFO:tensorflow:Converted 141 variables to const ops.
Model saved !!!
Batch: 583050   Current total loss: 6.8587523 pos_loss: 2.1344588 neg_loss: 1.4127902 loc_loss: 1.5821998
Batch: 583100   Current total loss: 15.509088 pos_loss: 5.9310136 neg_loss: 4.380931 loc_loss: 3.467343
Batch: 583150   Current total loss: 25.70348 pos_loss: 16.685863 neg_loss: 2.2566793 loc_loss: 5.0308294
Batch: 583200   Current total loss: 12.031036 pos_loss: 6.0607467 neg_loss: 1.5831392 loc_loss: 2.6565993
Batch: 583250   Current total loss: 13.24611 pos_loss: 4.6904907 neg_loss: 3.702229 loc_loss: 3.1224632
Batch: 583300   Current total loss: 24.90907 pos_loss: 14.844945 neg_loss: 2.7666862 loc_loss: 5.5661454
Batch: 583350   Current total loss: 13.186953 pos_loss

Batch: 586500   Current total loss: 9.871028 pos_loss: 3.7887027 neg_loss: 2.4486833 loc_loss: 1.8812914
INFO:tensorflow:Froze 141 variables.
INFO:tensorflow:Converted 141 variables to const ops.
Model saved !!!
Batch: 586550   Current total loss: 7.141938 pos_loss: 2.8065434 neg_loss: 1.1165503 loc_loss: 1.4660784
Batch: 586600   Current total loss: 14.0420685 pos_loss: 6.028817 neg_loss: 2.6805298 loc_loss: 3.5795815
Batch: 586650   Current total loss: 8.733111 pos_loss: 2.249216 neg_loss: 2.8504305 loc_loss: 1.8799305
Batch: 586700   Current total loss: 19.679325 pos_loss: 7.9657745 neg_loss: 4.3461304 loc_loss: 5.6134877
Batch: 586750   Current total loss: 14.397607 pos_loss: 6.6303244 neg_loss: 2.2545705 loc_loss: 3.7582927
Batch: 586800   Current total loss: 11.262775 pos_loss: 4.190299 neg_loss: 3.4876747 loc_loss: 1.8301938
Batch: 586850   Current total loss: 12.629864 pos_loss: 4.171298 neg_loss: 4.1683974 loc_loss: 2.535234
Batch: 586900   Current total loss: 8.936829 pos_los

INFO:tensorflow:Converted 141 variables to const ops.
Model saved !!!
Batch: 590050   Current total loss: 18.628487 pos_loss: 10.422678 neg_loss: 2.0950217 loc_loss: 4.335215
Batch: 590100   Current total loss: 6.923149 pos_loss: 1.6733885 neg_loss: 2.3148882 loc_loss: 1.1591591
Batch: 590150   Current total loss: 18.050993 pos_loss: 8.818588 neg_loss: 3.0316935 loc_loss: 4.424575
Batch: 590200   Current total loss: 12.395805 pos_loss: 4.9448047 neg_loss: 3.053671 loc_loss: 2.6208606
Batch: 590250   Current total loss: 16.283195 pos_loss: 7.1459017 neg_loss: 2.7536802 loc_loss: 4.6067863
Batch: 590300   Current total loss: 12.423489 pos_loss: 4.82164 neg_loss: 2.5077353 loc_loss: 3.3169136
Batch: 590350   Current total loss: 14.955635 pos_loss: 4.7088084 neg_loss: 4.2852225 loc_loss: 4.184059
Batch: 590400   Current total loss: 5.9958935 pos_loss: 1.620002 neg_loss: 1.2053912 loc_loss: 1.3926952
Batch: 590450   Current total loss: 25.44453 pos_loss: 12.965002 neg_loss: 3.684606 loc_los

...

Batch: 665250   Current total loss: 14.380915 pos_loss: 5.616983 neg_loss: 3.7524986 loc_loss: 2.8943343
Batch: 665300   Current total loss: 11.116924 pos_loss: 4.231985 neg_loss: 2.9579804 loc_loss: 1.8096905
Batch: 665350   Current total loss: 11.986959 pos_loss: 5.0378375 neg_loss: 2.0025506 loc_loss: 2.829187
Batch: 665400   Current total loss: 11.31293 pos_loss: 3.8266358 neg_loss: 2.97632 loc_loss: 2.3924162
Batch: 665450   Current total loss: 5.943321 pos_loss: 1.2774565 neg_loss: 1.2531875 loc_loss: 1.2950053
Batch: 665500   Current total loss: 8.085153 pos_loss: 2.0575438 neg_loss: 2.22514 loc_loss: 1.6844841
INFO:tensorflow:Froze 141 variables.
INFO:tensorflow:Converted 141 variables to const ops.
Model saved !!!
Batch: 665550   Current total loss: 7.6988363 pos_loss: 2.1604252 neg_loss: 2.3605924 loc_loss: 1.059625
Batch: 665600   Current total loss: 10.18911 pos_loss: 3.4468899 neg_loss: 2.3833191 loc_loss: 2.2404926
Batch: 665650   Current total loss: 8.88415 pos_loss: 3.8

Batch: 668850   Current total loss: 12.295795 pos_loss: 5.0176153 neg_loss: 3.364238 loc_loss: 1.7851951
Batch: 668900   Current total loss: 12.751005 pos_loss: 5.128288 neg_loss: 2.7794952 loc_loss: 2.714233
Batch: 668950   Current total loss: 11.361759 pos_loss: 4.976616 neg_loss: 1.9613273 loc_loss: 2.2946744
Batch: 669000   Current total loss: 5.402802 pos_loss: 1.131551 neg_loss: 1.3246434 loc_loss: 0.81751376
INFO:tensorflow:Froze 141 variables.
INFO:tensorflow:Converted 141 variables to const ops.
Model saved !!!
Batch: 669050   Current total loss: 3.9236696 pos_loss: 0.6804631 neg_loss: 0.5341303 loc_loss: 0.5797079
Batch: 669100   Current total loss: 9.564659 pos_loss: 4.1051264 neg_loss: 1.3421409 loc_loss: 1.9877964
Batch: 669150   Current total loss: 15.6190815 pos_loss: 8.000515 neg_loss: 2.009536 loc_loss: 3.479119
Batch: 669200   Current total loss: 7.47207 pos_loss: 1.7133522 neg_loss: 2.0040238 loc_loss: 1.6246254
Batch: 669250   Current total loss: 7.154803 pos_loss: 

Batch: 672400   Current total loss: 9.289585 pos_loss: 3.693255 neg_loss: 1.3392289 loc_loss: 2.1167247
Batch: 672450   Current total loss: 18.659864 pos_loss: 9.971174 neg_loss: 3.421823 loc_loss: 3.1262631
Batch: 672500   Current total loss: 11.894453 pos_loss: 5.6129484 neg_loss: 1.748811 loc_loss: 2.391968
INFO:tensorflow:Froze 141 variables.
INFO:tensorflow:Converted 141 variables to const ops.
Model saved !!!
Batch: 672550   Current total loss: 9.579799 pos_loss: 4.39462 neg_loss: 1.5656419 loc_loss: 1.4785725
Batch: 672600   Current total loss: 10.699744 pos_loss: 4.361663 neg_loss: 2.2546601 loc_loss: 1.9422257
Batch: 672650   Current total loss: 10.110657 pos_loss: 4.913518 neg_loss: 1.2785113 loc_loss: 1.7771438
Batch: 672700   Current total loss: 9.224985 pos_loss: 2.528509 neg_loss: 3.1011376 loc_loss: 1.4537779
Batch: 672750   Current total loss: 6.133624 pos_loss: 0.6336401 neg_loss: 2.4339328 loc_loss: 0.9244703
Batch: 672800   Current total loss: 8.648071 pos_loss: 2.72

Batch: 676000   Current total loss: 19.264643 pos_loss: 9.3825245 neg_loss: 2.9518955 loc_loss: 4.7770205
INFO:tensorflow:Froze 141 variables.
INFO:tensorflow:Converted 141 variables to const ops.
Model saved !!!
Batch: 676050   Current total loss: 10.935509 pos_loss: 4.604304 neg_loss: 2.0612261 loc_loss: 2.1167367
Batch: 676100   Current total loss: 8.490778 pos_loss: 3.2677414 neg_loss: 1.3353342 loc_loss: 1.7343266
Batch: 676150   Current total loss: 7.6293125 pos_loss: 1.6501316 neg_loss: 1.8166955 loc_loss: 2.0090203
Batch: 676200   Current total loss: 6.986546 pos_loss: 2.1785655 neg_loss: 1.1079156 loc_loss: 1.5465122
Batch: 676250   Current total loss: 9.007546 pos_loss: 3.4186623 neg_loss: 1.7036381 loc_loss: 1.731445
Batch: 676300   Current total loss: 12.997023 pos_loss: 4.1902304 neg_loss: 2.8222034 loc_loss: 3.8306427
Batch: 676350   Current total loss: 17.577635 pos_loss: 6.944024 neg_loss: 4.1317997 loc_loss: 4.3476834
Batch: 676400   Current total loss: 13.929387 pos_l

INFO:tensorflow:Froze 141 variables.
INFO:tensorflow:Converted 141 variables to const ops.
Model saved !!!
Batch: 679550   Current total loss: 13.270987 pos_loss: 5.1915627 neg_loss: 3.077582 loc_loss: 2.83781
Batch: 679600   Current total loss: 10.803629 pos_loss: 4.287072 neg_loss: 1.8258642 loc_loss: 2.5264766
Batch: 679650   Current total loss: 16.88945 pos_loss: 7.1783037 neg_loss: 3.4111626 loc_loss: 4.1356177
Batch: 679700   Current total loss: 8.107606 pos_loss: 2.3971543 neg_loss: 1.5820533 loc_loss: 1.9637251
Batch: 679750   Current total loss: 9.083773 pos_loss: 3.4469972 neg_loss: 2.0642693 loc_loss: 1.4075692
Batch: 679800   Current total loss: 9.875568 pos_loss: 3.8360038 neg_loss: 1.8263324 loc_loss: 2.0480924
Batch: 679850   Current total loss: 12.963694 pos_loss: 5.6212387 neg_loss: 2.1138325 loc_loss: 3.0633395
Batch: 679900   Current total loss: 16.344145 pos_loss: 6.087723 neg_loss: 6.16829 loc_loss: 1.9226913
Batch: 679950   Current total loss: 7.07448 pos_loss: 1.

In [ ]:
annotation = get_path_and_annotation('/media/xinje/New Volume/VOC07&12/VOC2007/train/train_OCR.txt')

In [ ]:
image_data, reorder_box,cls_data = get_random_data(annotation[7], (300,300), random=True, max_boxes=20, jitter=.3, hue=.1, sat=1.5, val=1.5, proc_img=True)

In [ ]:
plt.hlines(reorder_box[...,0],reorder_box[...,1],reorder_box[...,3],colors='red')
plt.hlines(reorder_box[...,2],reorder_box[...,1],reorder_box[...,3],colors='red')
plt.vlines(reorder_box[...,1],reorder_box[...,0],reorder_box[...,2],colors='red')
plt.vlines(reorder_box[...,3],reorder_box[...,0],reorder_box[...,2],colors='red')
plt.imshow(image_data)

In [ ]:
image_data.transpose(Image.FLIP_LEFT_RIGHT)

In [ ]:
image_data.shape

In [ ]:
image_data.transpose(0)

In [ ]:
reorder_box

In [ ]:
# def test_aug_func(image_batch,annotation_batch):
# ###show whether the image and the annotation are compatible after data augmentation
#     anno_batch = np.array(annotation_batch)*300
#     for index_img in range(FLAGS.batch_size):
#         for i in range(len(anno_batch[index_img])):
#             plt.hlines(int(anno_batch[index_img][i][0]),int(anno_batch[index_img][i][1]),int(anno_batch[index_img][i][3]),colors='red')
#             plt.hlines(int(anno_batch[index_img][i][2]),int(anno_batch[index_img][i][1]),int(anno_batch[index_img][i][3]),colors='red')
#             plt.vlines(int(anno_batch[index_img][i][1]),int(anno_batch[index_img][i][0]),int(anno_batch[index_img][i][2]),colors='red')
#             plt.vlines(int(anno_batch[index_img][i][3]),int(anno_batch[index_img][i][0]),int(anno_batch[index_img][i][2]),colors='red')
#         plt.imshow(image_batch[index_img])
#         plt.show()

In [ ]:
# def jaccard_with_anchors(bbox,ymin,xmin,ymax,xmax):
#     """Compute jaccard score between a box and the anchors.
#     """
#     int_ymin = np.maximum(ymin, bbox[0])
#     int_xmin = np.maximum(xmin, bbox[1])
#     int_ymax = np.minimum(ymax, bbox[2])
#     int_xmax = np.minimum(xmax, bbox[3])
#     h = np.maximum(int_ymax - int_ymin, 0.)
#     w = np.maximum(int_xmax - int_xmin, 0.)
#     # Volumes.
#     inter_vol = h * w
#     vol_anchors = (xmax - xmin) * (ymax - ymin)
#     union_vol = vol_anchors - inter_vol \
#         + (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
#     jaccard = np.divide(inter_vol, union_vol)
#     return jaccard

In [ ]:
# jaccard_with_anchors(anno_batch[0][0],ymin,xmin,ymax,xmax)

In [ ]:
# plt.figure(figsize=(20,20))
# plt.hlines(anno_batch[0][...,0]*300,anno_batch[0][...,1]*300,anno_batch[0][...,3]*300,colors='red')
# plt.hlines(anno_batch[0][...,2]*300,anno_batch[0][...,1]*300,anno_batch[0][...,3]*300,colors='red')
# plt.vlines(anno_batch[0][...,1]*300,anno_batch[0][...,0]*300,anno_batch[0][...,2]*300,colors='red')
# plt.vlines(anno_batch[0][...,3]*300,anno_batch[0][...,0]*300,anno_batch[0][...,2]*300,colors='red')

# plt.imshow(image_batch[0])